In [1]:
#git clone https://github.com/google-deepmind/alphafold3.git
pwd

/Users/jjaenes/24.11_alphafold_on_euler/batch-infer/software/alphafold3


In [2]:
cd alphafold3
docker buildx build --platform=linux/amd64 --progress plain -t alphafold3 -f docker/Dockerfile .
cd -

#0 building with "desktop-linux" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 2.73kB done
#1 DONE 0.0s

#2 [internal] load metadata for docker.io/nvidia/cuda:12.6.0-base-ubuntu22.04
#2 DONE 0.5s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [ 1/10] FROM docker.io/nvidia/cuda:12.6.0-base-ubuntu22.04@sha256:5dca947f477ec8ea91624447bdad7cc5f6a0dfe038ef53642c884bf5416478a6
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 14.28kB 0.0s done
#5 DONE 0.0s

#6 [ 6/10] COPY . /app/alphafold
#6 CACHED

#7 [ 3/10] RUN add-apt-repository ppa:deadsnakes/ppa     && DEBIAN_FRONTEND=noninteractive apt install --yes --quiet python3.11 python3-pip python3.11-venv python3.11-dev
#7 CACHED

#8 [ 4/10] RUN python3.11 -m venv /alphafold3_venv
#8 CACHED

#9 [ 5/10] RUN mkdir /hmmer_build /hmmer ;     wget http://eddylab.org/software/hmmer/hmmer-3.4.tar.gz --directory-prefix /hmmer_buil

In [3]:
docker image ls
#docker image rm 5b031e7d28fc
#mkdir -p images
docker save de9169bdbfeb -o images/alphafold3.tar

REPOSITORY   TAG       IMAGE ID       CREATED         SIZE
alphafold3   latest    de9169bdbfeb   3 minutes ago   9.06GB


In [ ]:
SRC='./images'
DEST='euler.ethz.ch:~/batch-infer/software/alphafold3/images'
rsync -azv --progress $SRC $DEST